In [3]:
import os
import numpy as np
import librosa
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score
import optuna
from tqdm import tqdm

optuna.logging.set_verbosity(optuna.logging.WARNING)

# 1. Настройки
ROOT_DATA_DIR = "1_dataset"

DATASETS = [
    os.path.join(ROOT_DATA_DIR, "20221011_dry_ds"),
    os.path.join(ROOT_DATA_DIR, "20221115_wet_ds")
]

POSSIBLE_PAV_TYPES = {"asphalt", "cobblestones"}

# 2. Функция извлечения признаков (один канал)
def extract_features(file_path, n_mfcc=13):
    try:
        y, sr = librosa.load(file_path, sr=None)

        features = []

        # MFCC + Delta + Delta-Delta
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        mfcc_d = librosa.feature.delta(mfcc)
        mfcc_dd = librosa.feature.delta(mfcc, order=2)

        for feat in [mfcc, mfcc_d, mfcc_dd]:
            features.extend(np.mean(feat, axis=1).tolist())
            features.extend(np.std(feat, axis=1).tolist())

        # Базовые спектральные признаки
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        zero_crossing = librosa.feature.zero_crossing_rate(y)[0]
        rms = librosa.feature.rms(y=y)[0]

        for arr in [spectral_centroids, zero_crossing, rms]:
            features.append(np.mean(arr))
            features.append(np.std(arr))

        return np.array(features)
    except Exception as e:
        print(f"Ошибка при обработке {file_path}: {e}")
        return None

# 3. Парсинг метки из имени файла
def parse_pav_type_from_filename(filename):
    parts = filename.replace(".wav", "").split("_")
    for part in parts:
        if part in POSSIBLE_PAV_TYPES:
            return part
    return None

# 4. Сбор данных из всех датасетов и каналов
print("Сбор данных из dry и wet датасетов...")

X = []
y = []

for dataset_dir in DATASETS:
    print(f"\nОбработка: {dataset_dir}")
    seg_dir_1 = os.path.join(dataset_dir, "sound_1", "segments_labeled")
    
    if not os.path.exists(seg_dir_1):
        print(f"Пропущен: {seg_dir_1} не найден")
        continue

    files = [f for f in os.listdir(seg_dir_1) if f.endswith(".wav")]
    print(f"  - Найдено {len(files)} сегментов")

    for file in tqdm(files, desc=f"  {os.path.basename(dataset_dir)}"):
        label = parse_pav_type_from_filename(file)
        if label is None:
            continue

        # Собираем признаки из 8 каналов
        all_feats = []
        valid = True
        for i in range(1, 9):
            seg_dir = os.path.join(dataset_dir, f"sound_{i}", "segments_labeled")
            full_path = os.path.join(seg_dir, file)
            if not os.path.exists(full_path):
                valid = False
                break
            feats = extract_features(full_path)
            if feats is None:
                valid = False
                break
            all_feats.append(feats)

        if valid:
            X.append(np.concatenate(all_feats))
            y.append(label)

X = np.array(X)
y = np.array(y)

print(f"\nВсего сегментов после объединения: {len(X)}")
print(f"Признаков на сегмент: {X.shape[1]}")
print(f"Уникальные метки: {np.unique(y)}")

# 5. Подготовка данных
le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

Сбор данных из dry и wet датасетов...

Обработка: 1_dataset/20221011_dry_ds
  - Найдено 834 сегментов


  20221011_dry_ds: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 834/834 [04:41<00:00,  2.97it/s]



Обработка: 1_dataset/20221115_wet_ds
  - Найдено 1325 сегментов


  20221115_wet_ds: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1325/1325 [07:27<00:00,  2.96it/s]


Всего сегментов после объединения: 2159
Признаков на сегмент: 672
Уникальные метки: ['asphalt' 'cobblestones']
Train: 1727, Test: 432


In [4]:
# 6. Optuna для SVM
def objective_svm(trial):
    C = trial.suggest_float("C", 1e-2, 1e2, log=True)
    gamma = trial.suggest_float("gamma", 1e-4, 1e0, log=True)

    model = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(
            C=C,
            gamma=gamma,
            kernel='rbf',
            class_weight='balanced',
            random_state=42
        ))
    ])

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='f1_weighted')
    return scores.mean()

# 7. Запуск Optuna
print("Подбор гиперпараметров для SVM:")
study = optuna.create_study(direction="maximize")
study.optimize(objective_svm, n_trials=50)

print("Лучшие параметры:", study.best_params)
print(f"Лучший F1 (CV): {study.best_value:.4f}")

# 8. Финальная модель
best_model = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(
        **study.best_params,
        kernel='rbf',
        class_weight='balanced',
        random_state=42
    ))
])

best_model.fit(X_train, y_train)

# 9. Прогноз на train и test
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

# 10. Отчёты
print("\n" + "="*70)
print("SVM (dry + wet, мультиканальный) — CLASSIFICATION REPORT (TRAIN)")
print("="*70)
print(classification_report(y_train, y_pred_train, target_names=le.classes_))

print("\n" + "="*70)
print("SVM (dry + wet, мультиканальный) — CLASSIFICATION REPORT (TEST)")
print("="*70)
print(classification_report(y_test, y_pred_test, target_names=le.classes_))

# F1 scores
f1_train = f1_score(y_train, y_pred_train, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')

print(f"\nF1-weighted (train): {f1_train:.4f}")
print(f"F1-weighted (test):  {f1_test:.4f}")

# Проверка переобучения
if f1_train - f1_test > 0.03:
    print("\nВозможное переобучение")
else:
    print("\nНет признаков значительного переобучения")

Подбор гиперпараметров для SVM:
Лучшие параметры: {'C': 3.4684554466506023, 'gamma': 0.00023310381598388974}
Лучший F1 (CV): 0.9830

SVM (dry + wet, мультиканальный) — CLASSIFICATION REPORT (TRAIN)
              precision    recall  f1-score   support

     asphalt       1.00      1.00      1.00      1676
cobblestones       0.88      1.00      0.94        51

    accuracy                           1.00      1727
   macro avg       0.94      1.00      0.97      1727
weighted avg       1.00      1.00      1.00      1727


SVM (dry + wet, мультиканальный) — CLASSIFICATION REPORT (TEST)
              precision    recall  f1-score   support

     asphalt       1.00      0.99      0.99       419
cobblestones       0.71      0.92      0.80        13

    accuracy                           0.99       432
   macro avg       0.85      0.96      0.90       432
weighted avg       0.99      0.99      0.99       432


F1-weighted (train): 0.9961
F1-weighted (test):  0.9870

Нет признаков значительно